In [3]:
from pathlib import Path
import json
import pandas as pd
from partitura.io.musescore import find_musescore
import subprocess

In [4]:
# open the generic tsv file in pandas
with open(Path(r"C:\Users\fosca\Desktop\JKU\romantic_piano_corpus\romantic_piano_corpus.metadata.tsv")) as f:
    romantic_df = pd.read_csv(f, sep='\t')
romantic_df.corpus.unique()

array(['beethoven_piano_sonatas', 'chopin_mazurkas',
       'debussy_suite_bergamasque', 'dvorak_silhouettes',
       'grieg_lyric_pieces', 'liszt_pelerinage', 'medtner_tales',
       'schumann_kinderszenen', 'tchaikovsky_seasons'], dtype=object)

In [5]:
romantic_df.head()

,corpus,piece,TimeSig,KeySig,last_mc,last_mn,length_qb,last_mc_unfolded,last_mn_unfolded,length_qb_unfolded,...,typesetter,text,pdf,score integrity,comments,staff_3_ambitus,staff_3_instrument,PDF,staff_4_ambitus,staff_4_instrument
0,beethoven_piano_sonatas,01-1,1: 2/2,1: -4,154,152,608.0,308.0,304.0,1216.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,beethoven_piano_sonatas,01-2,1: 3/4,1: -1,62,61,183.0,124.0,122.0,366.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,beethoven_piano_sonatas,01-3,1: 3/4,"1: -4, 43: -1",77,73,219.0,196.0,186.0,558.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,beethoven_piano_sonatas,01-4,1: 2/2,1: -4,199,196,790.0,392.0,390.0,1560.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,beethoven_piano_sonatas,02-1,1: 2/4,"1: 3, 127: 0, 230: 3",342,336,679.5,672.0,664.0,1336.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# open the debussy tsv file in pandas
with open(Path(r"C:\Users\fosca\Desktop\JKU\debussy_piano\concatenated_metadata.tsv")) as f:
    debussy_df = pd.read_csv(f, sep='\t')

In [7]:
debussy_df.head()

,corpus,fname,TimeSig,KeySig,last_mc,last_mn,length_qb,last_mc_unfolded,last_mn_unfolded,length_qb_unfolded,...,comment,annotated_key,harmony_version,annotators,reviewers,composed_source,imslp,viaf,musicbrainz,wikidata
0,debussy_childrens_corner,l113-01_childrens_doctor,1: 4/4,"1: 0, 33: -2, 37: -4, 45: 0",76,76,304.0,76,76,304.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,debussy_childrens_corner,l113-02_childrens_jimbos,1: 2/2,1: -2,81,81,324.0,81,81,324.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,debussy_childrens_corner,l113-03_childrens_serenade,1: 3/4,1: 4,124,124,372.0,124,124,372.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,debussy_childrens_corner,l113-04_childrens_snow,"1: 4/4, 48: 2/4, 49: 4/4, 52: 2/4, 53: 4/4",1: -1,74,74,292.0,74,74,292.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,debussy_childrens_corner,l113-05_childrens_little,1: 4/4,1: 3,31,31,124.0,31,31,124.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Merge the two datasets and convert to musicxml

We need to not duplicate debussy_suite_bergamasque

In [8]:
romantic_basepath = r"C:\Users\fosca\Desktop\JKU\romantic_piano_corpus"
debussy_basepath = r"C:\Users\fosca\Desktop\JKU\debussy_piano"

In [9]:
def convert_with_musescore(input_path, output_path, mscore_exec):
    cmd = [mscore_exec, "-o", output_path, input_path, "-f"]
    ps = subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.PIPE)
    if ps.returncode != 0:
        print("Command {} failed with code {}. MuseScore " "error messages:\n {}").format(cmd, ps.returncode, ps.stderr.decode("UTF-8"))

In [10]:
musescore = find_musescore()
print(musescore)
# add to path set PATH=%PATH%;C:\Program Files\MuseScore 4\bin\

# iterate through debussy_df
for i,row in debussy_df.iterrows():
    in_path = str(Path(debussy_basepath,row['corpus'],"MS3",row['fname']+".mscx"))
    out_path = str(Path(r"C:\Users\fosca\Desktop\JKU\piano_corpora_dcml\scores\debussy_corpus",row['fname']+".musicxml"))
    # add out folder path if it does not exist
    if not Path(out_path).parent.exists():
        Path(out_path).parent.mkdir(parents=True)
    # convert to musicxml using musescore
    convert_with_musescore(in_path,out_path,musescore)
    print("Piece converted",row['corpus'],row['fname'],out_path)

# iterate through romantic_df
for i,row in romantic_df.iterrows():
    # exclude debussy_suite_bergamasque since it is already included in the debussy corpus
    if row["corpus"]!= "debussy_suite_bergamasque":
        in_path = str(Path(romantic_basepath,row['corpus'],"MS3",row['piece']+".mscx"))
        out_path = str(Path(r"C:\Users\fosca\Desktop\JKU\piano_corpora_dcml\scores",row['corpus'],row['piece']+".musicxml"))
        # add out folder path if it does not exist
        if not Path(out_path).parent.exists():
            Path(out_path).parent.mkdir(parents=True)
        # convert to musicxml using musescore
        convert_with_musescore(in_path,out_path,musescore)
        print("Piece converted",row['corpus'],row['piece'],out_path)

C:\Program Files\MuseScore 4\bin\MuseScore4.exe
['C:\\Program Files\\MuseScore 4\\bin\\MuseScore4.exe', '-o', 'C:\\Users\\fosca\\Desktop\\JKU\\piano_corpora_dcml\\scores\\debussy_corpus\\l113-01_childrens_doctor.musicxml', 'C:\\Users\\fosca\\Desktop\\JKU\\debussy_piano\\debussy_childrens_corner\\MS3\\l113-01_childrens_doctor.mscx', '-f']
Piece converted debussy_childrens_corner l113-01_childrens_doctor C:\Users\fosca\Desktop\JKU\piano_corpora_dcml\scores\debussy_corpus\l113-01_childrens_doctor.musicxml
['C:\\Program Files\\MuseScore 4\\bin\\MuseScore4.exe', '-o', 'C:\\Users\\fosca\\Desktop\\JKU\\piano_corpora_dcml\\scores\\debussy_corpus\\l113-02_childrens_jimbos.musicxml', 'C:\\Users\\fosca\\Desktop\\JKU\\debussy_piano\\debussy_childrens_corner\\MS3\\l113-02_childrens_jimbos.mscx', '-f']
Piece converted debussy_childrens_corner l113-02_childrens_jimbos C:\Users\fosca\Desktop\JKU\piano_corpora_dcml\scores\debussy_corpus\l113-02_childrens_jimbos.musicxml
['C:\\Program Files\\MuseScore 4